In [2]:
#classical naive bayas probability model for baseline

from sklearn.naive_bayes import GaussianNB, CategoricalNB


X_train = train_df.drop(columns=["woningtype"])
y_train = train_df["woningtype"]

X_val = val_df.drop(columns=["woningtype"])
y_val = val_df["woningtype"]

X_test = test_df.drop(columns=["woningtype"])
y_test = test_df["woningtype"]

#wow moeilijke code (niet)
#guassian NB ipv categorical omdat onze data niet alleen categorical is (WOW)
NB = GaussianNB()
NB.fit(X_train, y_train)

NameError: name 'train_df' is not defined

In [3]:
#discriminitive classical model (probabky xgboost or random forest)
#random forest is denk beter als deze modellen alleen als baseline worden gebruikt
#xg is meer accurate tho
#heb de categorical features niet geencode want dacht dat koen dat al gedaan had? anders ga ik er nog achteraan

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

X_train = train_df.drop(columns=["woningtype"])
y_train = train_df["woningtype"]

X_val = val_df.drop(columns=["woningtype"])
y_val = val_df["woningtype"]

X_test = test_df.drop(columns=["woningtype"])
y_test = test_df["woningtype"]

#random forest is minder accuraat maar meer interpeteerbaar en minder complex
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

#accurater maar moet ff meer hyperparamter tuningg doen
xgb_clf = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)
xgb_clf.fit(X_train, y_train)

#Ik heb nog even niets aangeraakt voor het valideren want weet nog niet welke metric we gebruiken
#wil ook niet onze test zet verneuken en leake ofzo
#moet nog parameters tunen maar wilde de pipeline hiervoor gebruiken denk ik?



NameError: name 'train_df' is not defined

In [ ]:
import numpy as np
import torch
import seaborn as sns

from sklearn.metrics import (
        accuracy_score, precision_score, recall_score, f1_score,
        log_loss, cohen_kappa_score, 
    )

def evaluate_model(model, dataloader, device, class_names=None):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, tabular_data, labels in dataloader:
            images = images.to(device)
            tabular_data = tabular_data.to(device)
            labels = labels.to(device)
    
            outputs = model(images, tabular_data)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            preds = np.argmax(probs, axis=1)
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    kappa = cohen_kappa_score(all_labels, all_preds)
    logloss = log_loss(all_labels, all_probs)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1 Score (macro): {f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")
    print(f"Log Loss: {logloss:.4f}")